In [1]:
from elasticsearch import Elasticsearch
import pandas as pd

In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import numpy as np
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
import sys
import os

In [12]:
sys.path.append("../")
from evaluation.metrics import hit_k, mean_hit_k, average_precision_k, mean_average_precision_k
from utils.constants import TRANSCRIPT_COL
from utils.helpers import extract_query_from_df
import time
from tqdm.auto import tqdm
from sentence_transformers import util

In [11]:
model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
max_seq_length=128
model = SentenceTransformer(model_name)
model.max_seq_length = max_seq_length

c:\Users\Lenovo\anaconda3\envs\pytorch\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [35]:
username = 'elastic'
pwd = 'password'
index_name = 'transcript'

In [43]:
# client = Elasticsearch("http://localhost:9200")
client = Elasticsearch("https://localhost:9200",ca_certs="ca.crt",basic_auth=("elastic", "password"))

In [44]:
client.info()

ObjectApiResponse({'name': 'es01', 'cluster_name': 'docker-cluster', 'cluster_uuid': '43w2srkGTOOeCr5P57l5Hg', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [18]:
for sub in ['optim','signal','stat']:
    print(os.listdir(f'../data/all/{sub}'))

['optim1.csv', 'optim2.csv', 'optim3.csv', 'optim4.csv']
['signal1.csv', 'signal10.csv', 'signal2.csv', 'signal3.csv', 'signal4.csv', 'signal5.csv', 'signal6.csv', 'signal7.csv', 'signal8.csv', 'signal9.csv']
['stat1.csv', 'stat2.csv', 'stat3.csv', 'stat4.csv', 'stat5.csv', 'stat6.csv', 'stat7.csv', 'stat8.csv']


In [ ]:
# id = 0
# index_name = 'transcripts'
# for sub in ['optim','signal','stat']:
#     for file in os.listdir(f'../data/all/{sub}'):
#         filename = f"../data/all/{sub}/{file}"
#         df = pd.read_csv(filename, index_col=0)
#         query_dict = extract_query_from_df(df)
#         corpus_embeddings = df[TRANSCRIPT_COL]
#         corpus_embeddings = model.encode(
#             corpus_embeddings, convert_to_tensor=False, show_progress_bar=True
#             )
#         df['embeddings'] = list(corpus_embeddings)
#         df['clip_id'] = id
#         id+=1
#         documents = df.to_dict(orient='records')
#         # Index documents into Elasticsearch
#         for document in documents:
#             # Index document into Elasticsearch
#             client.index(index=index_name, body=document)

In [53]:
resp = client.search(index="transcripts", query={"match_all": {}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

Got 3190 Hits:


KeyError: 'timestamp'

In [55]:
query_list = [
    (5,'การ convolution'),
    (10,'การบ้าน'),
    (8,'linear optimization'),
    (4,'fourier transform'),
    (0,'aliasing'),
    (2,'integer programming'),
    (7,'signal สอบอะไร'),
    (6,'epsilon greedy'),
    (1,'power'),
    (9,'optimization')
]

In [83]:
search_time = []
for id,query in query_list:
    query_embedding = model.encode(query, convert_to_tensor=False)
    query = {
            "knn": {
                "field": "embeddings",
                "query_vector": query_embedding,
                "num_candidates": 20,
                "filter": {
                    "term" : { "clip_id" : id }
                }
            }
        }
    start_time = time.time()
    resp = client.search(index="transcripts",size=10, query=query)
    end_time = time.time()
    search_time.append(end_time - start_time)
    # for hit in resp['hits']['hits']:
    #     print(hit["_source"]["text"])